In [1]:
%load_ext Cython

In [2]:
import matplotlib.pyplot as pyplot
import optuna as op
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Utils.createICM import newCreateICM
from Utils.createURM import createURM
from Utils.createICM import createICM
from Utils.combineURMICM import combine

In [3]:
URM = createURM()
ICM = createICM()

C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Utils\createURM.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('../Input/interactions_and_impressions.csv')
C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Utils\createICM.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  URM = pd.read_csv('../Input/interactions_and_impressions.csv')


In [ ]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

In [ ]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [ ]:
URMICMCombined = combine(ICM, URM_train.T)

## Neighbors

In [ ]:
x_tick = [10, 25, 40, 50, 100]
MAP_per_k = []

for topK in x_tick:

    recommender = ItemKNNCFRecommender(URM_train)
    recommender.fit(shrink=0.0, topK=topK)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender)

    MAP_per_k.append(result_df.loc[10]["MAP"])

In [ ]:
pyplot.plot(x_tick, MAP_per_k)
pyplot.ylabel('MAP')
pyplot.xlabel('TopK')
pyplot.show()

## Shrink

In [ ]:
x_tick = [0, 10, 50, 100, 200]
MAP_per_shrinkage = []

for shrink in x_tick:

    recommender = ItemKNNCFRecommender(URM_train)
    recommender.fit(shrink=shrink, topK=45)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender)

    MAP_per_shrinkage.append(result_df.loc[10]["MAP"])

In [ ]:
pyplot.plot(x_tick, MAP_per_shrinkage)
pyplot.ylabel('MAP')
pyplot.xlabel('Shrinkage')
pyplot.show()

## OPTUNA optimization

In [16]:
# objective = represents the function that we need to minimize
# trial = optuna parameter that tries random values in a certain interval
recommender = ItemKNNCFRecommender(URMICMCombined.T)
def objective(trial):
    topK = trial.suggest_int("topK", 60, 80)
    shrink = trial.suggest_float("shrink", 175, 200)
    recommender.fit(topK=topK, shrink=shrink)
    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    return result_dict.loc[10]["MAP"]

In [17]:
study = op.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2022-11-06 10:57:01,123] A new study created in memory with name: no-name-f8d02c20-7ec5-47cb-9606-987152a39d80


Similarity column 24507 (100.0%), 7097.09 column/sec. Elapsed time 3.45 sec


C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Evaluation\metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Evaluation\metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40856 (100.0%) in 24.66 sec. Users per second: 1657


[I 2022-11-06 10:57:29,337] Trial 0 finished with value: 0.006875244762091309 and parameters: {'topK': 65, 'shrink': 182.48100349542025}. Best is trial 0 with value: 0.006875244762091309.


...

Similarity column 24507 (100.0%), 6475.40 column/sec. Elapsed time 3.78 sec


C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Evaluation\metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Evaluation\metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40856 (100.0%) in 22.34 sec. Users per second: 1829


[I 2022-11-06 10:59:40,129] Trial 5 finished with value: 0.0069022492082141396 and parameters: {'topK': 68, 'shrink': 199.7290233898909}. Best is trial 1 with value: 0.006932146120637478.


KeyboardInterrupt: 

In [ ]:
topK = study.best_params['topK']
shrink = study.best_params['shrink']

In [6]:
recommender = ItemKNNCFRecommender(URM_train)
recommender.fit(shrink=shrink, topK=topK)
result_df, _ = evaluator_test.evaluateRecommender(recommender)
result_df

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 24507 (100.0%), 809.83 column/sec. Elapsed time 30.26 sec
EvaluatorHoldout: Processed 41308 (100.0%) in 27.70 sec. Users per second: 1491


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.051922,0.06987,0.060524,0.024824,0.033011,0.162364,0.075162,0.055894,0.335649,0.199303,...,0.992289,0.333061,0.992289,0.102962,9.910954,0.994396,0.204605,0.734815,3.470055,0.351586
